## Fetch Data From Property Assessment  

### Get Database id by year

In [55]:
import data.sqlDataFetch as sdf

In [202]:
database_id = sdf.get_id_by_year(2009)

### Describe The Database

Because of each database have diff key value, I can not use for loop to get key value

In [203]:
sdf.describe_database(database_id)

Column Name          | Type      
--------------------------------
_id                  | int       
PID                  | text      
CM_ID                | text      
ST_NUM               | text      
ST_NAME              | text      
ST_NAME_SUF          | text      
UNIT_NUM             | text      
ZIPCODE              | text      
PTYPE                | text      
LU                   | text      
OWN_OCC              | text      
OWNER                | text      
MAIL_ADDRESS         | text      
MAIL CS              | text      
MAIL_ZIPCODE         | text      
AV_LAND              | text      
AV_BLDG              | text      
AV_TOTAL             | text      
GROSS_TAX            | text      
LAND_SF              | text      
YR_BUILT             | text      
YR_REMOD             | text      
GROSS_AREA           | text      
LIVING_AREA          | text      
NUM_FLOORS           | text      
STRUCT_CLS           | text      
R_BLDG_STYL          | text      
R_ROOF_TYP     

In [204]:
import requests
import pandas as pd

### Get Con_unit, Res_unit, Rc_unit for each year

In [205]:
url = "https://data.boston.gov/api/3/action/datastore_search_sql"
# Construct the SQL query to select distinct city names from the table
sql_query = f"""
SELECT
    "ZIPCODE"::text AS ZIPCODE,
    SUM("S_UNIT_RES"::INT) AS total_res_units,
    SUM("S_UNIT_COM"::INT) AS total_com_units,
    SUM("S_UNIT_RC"::INT) AS total_rc_units
FROM
    "{database_id}"
GROUP BY
    "ZIPCODE"::text
"""
print(sql_query)
params = {"sql": sql_query}

# Send the request
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()

    # Check if there is data in the response
    if data['success'] and 'result' in data and 'records' in data['result']:
        records = data['result']['records']
        # Filter records with ZIP code length greater than 1 and ensure ZIPCODE is not None
        filtered_records = [record for record in records if record['zipcode'] and len(record['zipcode']) > 1]

        # Convert the filtered records into a DataFrame
        df = pd.DataFrame(filtered_records)
    else:
        print("No data found or error in response.")
else:
    print("Failed to fetch data:", response.status_code)



SELECT
    "ZIPCODE"::text AS ZIPCODE,
    SUM("S_UNIT_RES"::INT) AS total_res_units,
    SUM("S_UNIT_COM"::INT) AS total_com_units,
    SUM("S_UNIT_RC"::INT) AS total_rc_units
FROM
    "1a374bd0-1ff9-4d1a-8727-ddfc201254fe"
GROUP BY
    "ZIPCODE"::text



In [206]:
#df['zipcode'] = df['zipcode'].str.replace('_', '', regex=False)
print(df)

   zipcode  total_res_units  total_com_units  total_rc_units
0    02090              NaN              NaN             NaN
1    02108           1031.0            216.0             2.0
2    02109           1199.0             84.0             8.0
3    02110            872.0            273.0             0.0
4    02111           1635.0            154.0           285.0
5    02113           1132.0             38.0            28.0
6    02114           3334.0            105.0             2.0
7    02115           2806.0             47.0             3.0
8    02116          15977.0            362.0           173.0
9    02118           6709.0            199.0            25.0
10   02119           1321.0             36.0             7.0
11   02120            475.0              5.0             0.0
12   02121            402.0              3.0             0.0
13   02122           1384.0              6.0            10.0
14   02124           1628.0             32.0            54.0
15   02125           165

### save each cvs files

In [196]:
import os

save_path = './data/cleaned/'
file_name = 'property_assessment_2009.csv'
full_path = os.path.join(save_path, file_name)

if not os.path.exists(save_path):
  os.makedirs(save_path)

df.to_csv(full_path, index=False)